In [301]:
#Linear Regression
#https://www.tensorflow.org/tutorials/keras/regression

from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

1.14.1-dev20190430


In [302]:
dataset_path="merged.csv"
column_names = ['idx','Menu Item Number','Menu Item','Sales Count','Sales Total', 
                'Date', 'Day', 'Name of Restaurant']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?",dtype={'idx': object},
                      sep=",", skiprows=[0])
raw_dataset.drop(['idx'], axis=1)
dataset = raw_dataset.copy()
#find unknown values
dataset.isna().sum()
#drop rows with unknown values
dataset = dataset.dropna()
#dataset.tail(30)
dataset.head()

,idx,Menu Item Number,Menu Item,Sales Count,Sales Total,Date,Day,Name of Restaurant
0,7,2429,TO GO,1,0,2019-03-18,0,Origami
1,8,3624,SALMON POKE BOWL,4,29,2019-03-18,0,Origami
2,9,3630,SPCY CRAB MAKI,17,114.75,2019-03-18,0,Origami
3,10,3631,SALM CR CHZ MAKI,8,55.6,2019-03-18,0,Origami
4,11,3632,TUNA MAKI,2,13.5,2019-03-18,0,Origami


In [303]:
#useless idx col
dataset.pop('idx')
#manually clean errors in dataset :(
#day column is string, convert it to int 
dataset['Sales Count'] = dataset['Sales Count'].astype(int)
dataset['Sales Total'] = dataset['Sales Total'].astype(float) #encountered '$-' in csv 115610, manually changed to 0
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Day'] = dataset['Day'].astype(int)

#Day column is categorical not numeric
day = dataset.pop('Day')
# print(day.dtypes)
dataset['Mon'] = (day == 0)*1.0
dataset['Tues'] = (day == 1)*1.0
dataset['Wed'] = (day == 2)*1.0
dataset['Thurs'] = (day == 3)*1.0
dataset['Fri'] = (day == 4)*1.0
dataset['Sat'] = (day == 5)*1.0
dataset['Sun'] = (day == 6)*1.0

dataset['Year'] = pd.DatetimeIndex(dataset['Date']).year
dataset['Month'] = pd.DatetimeIndex(dataset['Date']).month

In [304]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
#what info would a joint distribution give me for this?
#sns.pairplot(train_dataset[['Sales Count','Sales Total']], diag_kind="kde")


In [305]:
# train_stats = train_dataset.describe()
# train_stats.pop("Sales Count")
# train_stats = train_stats.transpose()
# train_stats

In [306]:
# train_stats = train_dataset.describe()
# train_stats.pop("Sales Total")
# train_stats = train_stats.transpose()
# train_stats

In [307]:
#Separate the target value, or "label", from the features. 
#This label is the value that you will train the model to predict.
train_labels = train_dataset.pop('Sales Count')
test_labels = test_dataset.pop('Sales Count')

In [308]:
def build_model():
    #call initializer instance with the dtype argument instead of passing it to the constructor error
    #deprecated keras.Sequential, change to tf.keras.Sequential
    model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model
model = build_model()
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 64)                960       
_________________________________________________________________
dense_28 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 65        
Total params: 5,185
Trainable params: 5,185
Non-trainable params: 0
_________________________________________________________________


In [309]:
example_batch = train_dataset[:10]
print(example_batch)
example_result = model.predict(example_batch)
example_result

       Menu Item Number        Menu Item  Sales Total       Date  \
2130               3632        TUNA MAKI       121.50 2018-09-10   
242222             5814   BRD TRK-HM-RBF        33.95 2017-03-30   
112143             3534  YTG FRT PINAPLE       169.00 2017-04-05   
159005             8222       ICED TEA M        26.00 2018-03-14   
211170             1953             EGGS         2.00 2019-04-01   
109645             2297  SPECIAL CAKE TP        11.25 2018-11-02   
57703              6322   STRW SURFRDR M       106.02 2016-10-13   
168919             4806        3 T STEAK        87.75 2018-10-26   
184595             4858   SIDE GUACAMOLE        21.60 2017-02-08   
89477              2314    LORRAINE S CR        57.75 2018-12-12   

       Name of Restaurant  Mon  Tues  Wed  Thurs  Fri  Sat  Sun  Year  Month  
2130              Origami  1.0   0.0  0.0    0.0  0.0  0.0  0.0  2018      9  
242222   Brueggers Bagels  0.0   0.0  0.0    1.0  0.0  0.0  0.0  2017      3  
112143      Do

ValueError: could not convert string to float: 'TUNA MAKI'